In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re

In [3]:
#https://www.kaggle.com/muonneutrino/wikipedia-traffic-data-exploration
#fill nan values with 0 upon importing data, since data doesn't distinguish between 0 and missing
#to save memory, downcast data to integers (keep in mind predictions will prob be floating point)
#parse out the language of each url, and plot total views by language
#some of these won't have a language, so label them as na
#plot each language on its own, show fourier transform for each
#plot views for some random pages
#for each language, find the top visited pages
#create ARIMA predictions for pages wth very high viewcount

In [4]:
train = pd.read_csv('train_1.csv')
key = pd.read_csv('key_1.csv')
submission = pd.read_csv('sample_submission_1.csv')

In [5]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [6]:
def find_lang(page):
    match = re.search('[a-z][a-z].wikipedia.org', page)
    if match:
        return match.group(0)[0:2]
    return 'na'

def find_article(page):
    match = re.search('[a-z][a-z].wikipedia.org.', page)
    if match:
        article_end = match.start()-1
        return(page[0:article_end])
    return 'na'

def find_access(page):
    match = re.search('.*.wikipedia.org.', page)
    if match:
        access_start = match.end()+1
        return(page[access_start:])
    return 'na'

train['Language'] = train['Page'].map(find_lang)
train['Article'] = train['Page'].map(find_article)
train['Access'] = train['Page'].map(find_access)

In [52]:
#fill nan with 0 since the dataset doesn't differentiate between missing and 0's
dates = train.iloc[:,1:551].fillna(0)
labels = train[['Article','Access','Page']]

for col in dates:
    dates[col] = pd.to_numeric(dates[col], downcast = 'integer')

In [53]:
newtrain = pd.concat([labels, dates], axis = 1)

In [54]:
newtrain.head()

,Article,Access,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1,ll-access_spider,2NE1_zh.wikipedia.org_all-access_spider,18,11,5,13,14,9,9,...,32,63,15,26,14,20,22,19,18,20
1,2PM,ll-access_spider,2PM_zh.wikipedia.org_all-access_spider,11,14,15,18,11,13,22,...,17,42,28,15,9,30,52,45,26,20
2,3C,ll-access_spider,3C_zh.wikipedia.org_all-access_spider,1,0,1,1,0,4,0,...,3,1,1,7,4,4,6,3,4,17
3,4minute,ll-access_spider,4minute_zh.wikipedia.org_all-access_spider,35,13,10,94,4,26,14,...,32,10,26,27,16,11,17,19,10,11
4,52_Hz_I_Love_You,ll-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0,0,0,0,0,0,0,...,48,9,25,13,3,11,27,13,36,10


In [55]:
newtrain.groupby('Language').sum()

KeyError: 'Language'

In [ ]:
newtrain.info()